In [ ]:
%%bash
# install dependencies
apt install -y autoconf2.13 autotools-dev bison flex g++ gettext git imagemagick libblas-dev libbz2-dev libcairo2-dev libfftw3-dev libfreetype6-dev libgdal-dev libgeos-dev libglu1-mesa-dev libjpeg-dev liblapack-dev liblas-c-dev libncurses5-dev libnetcdf-dev libpng-dev libpq-dev libproj-dev libreadline-dev libsqlite3-dev libtiff-dev libxmu-dev libzstd-dev make netcdf-bin p7zip proj-bin sqlite3 unixodbc-dev xvfb zlib1g-dev libgomp1 subversion parallel

pip install Pillow
pip install ply
pip install PyVirtualDisplay

git clone --depth 1 https://github.com/OSGeo/grass.git grass-src

# enter the directory with source code
cd grass-src

# compile
./configure \
    --enable-largefile=yes \
    --with-nls \
    --with-cxx \
    --with-readline \
    --with-bzlib \
    --with-pthread \
    --with-proj-share=/usr/share/proj \
    --with-geos=/usr/bin/geos-config \
    --with-cairo \
    --with-opengl-libs=/usr/include/GL \
    --with-freetype=yes --with-freetype-includes="/usr/include/freetype2/" \
    --with-sqlite=yes \
    --with-openmp
make -j2
make -j2 install

# leave the directory with source code
cd ~

# download sample data
mkdir -p grassdata
curl -SL https://grass.osgeo.org/sampledata/north_carolina/nc_spm_08_grass7.zip > nc_spm_08_grass7.zip
unzip -qq nc_spm_08_grass7.zip
mv nc_spm_08_grass7 grassdata
rm nc_spm_08_grass7.zip
wget https://zenodo.org/record/6967699/files/FOSS4G_2022_GRASS_GIS_workshop.zip
unzip FOSS4G_2022_GRASS_GIS_workshop.zip
rm FOSS4G_2022_GRASS_GIS_workshop.zip

In [ ]:
import os
os.chdir(os.path.expanduser("~"))

# Part 1: GRASS GIS intro

In this first part, we will demonstrate starting GRASS GIS, creating new project and basic data visualization.

## Notebook environment

By default all cells are running Python:

In [ ]:
import sys
v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

Cells can also run bash using IPython magic. Later, we will use this to show how GRASS can be used from shell.

In [ ]:
%%bash
pwd

or we can use ! to run individual lines in bash:

In [ ]:
!pwd

## Start GRASS GIS

First, we create new empty location (project) called *dix_park* that uses projection [UTM zone 17 N](https://epsg.io/6346) with EPSG:6346.

Flag `c` stands for _creating_ new location and `e` will _exit_ the command after creating the location. See [manual](https://grass.osgeo.org/grass-stable/manuals/grass.html) for more examples.

In [ ]:
%%bash
grass -c EPSG:6346 -e ~/grassdata/dix_park/ 

Start GRASS GIS session in the newly created location. Load Python libraries.

In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
session = gj.init("~/grassdata", "dix_park", "PERMANENT")

## Import data

_If you run this workshop online, data is already prepared in the environment. Otherwise download the data [here](http://fatra.cnr.ncsu.edu/foss4g2021/)._

We will import prepared digitial surface model (DSM), bare ground (digital terrain model, DTM) and ortho maps. 
The data CRS matches the CRS of the *dix_park* location, so we don't need to reproject it.

In [ ]:
%%bash
r.import input=dsm.tif output=dsm resample=bilinear
r.import input=ground.tif output=ground
r.import input=ortho.tif output=ortho

Next, we will import pre-downloaded OSM data of roads restricted to our study area. We obtained the roads using Overpass Turbo with this [query](https://overpass-turbo.eu/s/1aGu) and exported to GeoJSON. 

The data comes in EPSG:4326, so it will be automatically reprojected to UTM during the import.

In [ ]:
%%bash
v.import input=roads.geojson output=roads

Let's look at the available data in our location:

In [ ]:
%%bash
g.list type=raster,vector -m -t

## Data visualization

We will call d.rast/d.vect modules using the new [GRASS Jupyter API](https://grass.osgeo.org/grass-stable/manuals/libpython/grass.jupyter.html). The `Map` class creates and displays GRASS maps as static PNG images.

First let's display bare ground elevation with the roads dataset:

In [ ]:
# Create Map instance
elevation_map = gj.Map()
# Add a raster and vector to the map
elevation_map.d_rast(map="ground")
elevation_map.d_vect(map="roads")
elevation_map.d_legend(raster="ground")
# Display map
elevation_map.show()

Next, display road _Umstead Drive_ in yellow on top of ortho. Method `d_vect` can be called with options of [d.vect](https://grass.osgeo.org/grass-stable/manuals/d.vect.html).

In [ ]:
ortho_map = gj.Map()
ortho_map.d_rast(map="ortho")
ortho_map.d_vect(map="roads", width="2", color="yellow", where="name = 'Umstead Drive'")
ortho_map.show()

Here is how we can visualize data interactively with folium:

In [ ]:
roads_map = gj.InteractiveMap()
roads_map.add_raster("dsm", opacity=0.5)
roads_map.add_vector("roads")
roads_map.add_layer_control(position="bottomright")
roads_map.show()

We can also visualize data in 3D. Here we drape the ortho over the DSM.


In [ ]:
map3d = gj.Map3D()
map3d.render(elevation_map="dsm", color_map="ortho",
             position=(0.5, 1), height=3000, perspective=12)
map3d.show()

In [ ]:
map3d = gj.Map3D()
map3d.render(elevation_map="dsm", resolution_fine=1, color_map="ortho",
             light_position=(1, 0, 0.5),
             position=(0.75, 0.35), height=1500, perspective=10)
map3d.show()

## GRASS GIS tools

GRASS functionality is available through tools (also called modules). There are over 500 different tools in the core distribution and over 300 addon tools that can be used to prepare and analyze data.

Tools respect the following naming conventions:

Prefix | Function | Example
------ | -------- | -------
r.* | raster processing | r.mapcalc: map algebra
v.*	| vector processing	| v.clean: topological cleaning
i.*	| imagery processing | i.segment: object recognition
db.* | database management | db.select: select values from table
r3.* | 3D raster processing | r3.stats: 3D raster statistics
t.* | temporal data processing | t.rast.aggregate: temporal aggregation
g.* | general data management | g.rename: renames map
d.* | display | d.rast: display raster map

Note also that some tools have multiple dots in their names. For example, tools staring with v.net.* deal with vector network analysis and r.in.* tools import raster data into GRASS GIS spatial database.

There is a tool for finding other tools:

In [ ]:
%%bash
g.search.modules keyword=zonal

Here is how to get all options and flags of a GRASS tool:

In [ ]:
%%bash
r.univar --help

This will open the tool's manual page. Remove -m flag to open HTML page in your browser (will work only locally).

In [ ]:
%%bash
g.manual r.univar -m

GRASS modules can be executed either through the GUI, command line or Python interfaces. This is an example how to execute a tool in command line. Specifically, it will extract road _Umstead Drive_ into a new vector `umstead_drive_segments`.

In [ ]:
%%bash
v.extract input=roads where="name = 'Umstead Drive'" output=umstead_drive_segments

Now the same in Python:

In [ ]:
gs.run_command("v.extract", input="roads", where="name = 'Umstead Drive'", output="umstead_drive_segments")

### Computational region

Computational region is an important raster concept in GRASS GIS.
Before we use a module to compute a new raster map, we need to properly set the computational region. All raster computations will be performed in the specified extent and with the given resolution.
Among other things, this allows us to easily subset larger extent data for quicker testing of analysis, or to run an analysis of specific regions given by e.g. administrative units.

A few points to keep in mind:

 * computational region is defined by region extent and raster resolution
 * applies to all raster operations and vector to raster operations
 * persists between GRASS sessions, can be different for different mapsets
 * advantages: keeps your results consistent, avoids clipping, for computationally demanding tasks set region to smaller extent, check that your result is good and then set the computational region to the entire study area and rerun analysis
 * run `g.region -p` to see current region settings
 

In [ ]:
%%bash
g.region -p

The most common way to set region is **based on a raster map** - both extent and resolution. Run again g.region (we include -p flag to always see the resulting region):

In [ ]:
%%bash
g.region -p raster=dsm

Computational region can be set also **using a vector map**. In that case, only extent is set (as vector maps do not have any resolution - at least not in the way raster maps do). In the command line, it looks like this:


In [ ]:
%%bash
g.region -p vector=roads | tail

However now the resolution was adjusted based on the extent of the vector map, it is no longer a nice rounded number. If that's not desired, we can set it explicitly using -a flag and parameter res. Now the resolution is aligned to even multiples of 2 (the units are the units of the current location, in our case meters):


In [ ]:
%%bash
g.region -p vector=roads res=2 -a

Often we need to set the computational extent based on a vector map, but take the resolution and alignment from a raster map:

In [ ]:
%%bash
g.region -p vector=roads align=dsm

Finally, we can save a specific region for later. This won't actually modify the current region:

In [ ]:
%%bash
g.region vector=roads align=dsm save=roads_region

If we want to later set that named region, we can do it simply with:

In [ ]:
%%bash
g.region region=roads_region -p

## Python API

There are two Python APIs for accessing a tool's functionality - [GRASS GIS Python Scripting Library](https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html) and [PyGRASS](https://grass.osgeo.org/grass-stable/manuals/libpython/pygrass_index.html).
PyGRASS is advantageous for more advanced workflows. Here we will be using Python Scripting Library (`import grass.script as gs`)
as it is simple and straightforward to use.
 

GRASS GIS Python Scripting Library provides functions to call GRASS modules within scripts as subprocesses. The most often used functions include:

 * [run_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.run_command): used with modules which output raster/vector data where text output is not expected
 * [read_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.read_command): used when we are interested in text output
 * [parse_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.parse_command): used with modules producing text output as key=value pair
 * [write_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.write_command): for modules expecting text input from either standard input or file


It also provides several wrapper functions for often called modules. The list of convenient wrapper functions with examples includes:

 * Raster metadata using [raster_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.raster_info): `gs.raster_info('dsm')`
 * Vector metadata using [vector_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.vector.vector_info): `gs.vector_info('roads')`
 * List raster data in current location using [list_grouped()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.list_grouped): `gs.list_grouped(type=['raster'])`
 * Get current computational region using [region()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.region): `gs.region()`
 * Run raster algebra using [mapcalc()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.mapcalc): `gs.mapcalc()`


Let's use Python to compute viewshed:

In [ ]:
# 1. set computational region based on DSM
gs.run_command("g.region", raster="dsm")
# 2. Compute viewshed, flag 'b' is for binary (0 and 1) output 
gs.run_command("r.viewshed", input="dsm", output="viewshed", flags="b", coordinates=(711260, 3960860))
# 3. Compute basic univariate statistics, flag 'g' is for parsable output
univar = gs.parse_command("r.univar", map="viewshed", flags='g')
# 4. Get current region settings to get cell size
region = gs.region()
cell_size = region["nsres"] * region["ewres"]
percentage = 100 * float(univar['sum']) / float(univar['n'])
area = cell_size * float(univar['sum'])
print(f"Percentage of visible area is {percentage:.2f}%, which is {area / 10000:.2f} ha")

In [ ]:
viewshed_map = gj.Map()
viewshed_map.d_rast(map="ortho")
# select only cells with value 1 to visualize
viewshed_map.d_rast(map="viewshed", values=1)
viewshed_map.show()

# Part 2: Viewshed case study

In the second part, we will demonstrate the use of GRASS for a small viewshed case study.
The goal is to **compute and analyze the area a driver would see from a road**.
This notebook can be run only after notebook Part 1 was executed.

Topics covered:
 * Python scripting
 * manipulating vector data ([v.build.polylines](https://grass.osgeo.org/grass-stable/manuals/v.build.polylines.html), [v.to.points](https://grass.osgeo.org/grass-stable/manuals/v.to.points.html))
 * vector attributes ([v.db.select](https://grass.osgeo.org/grass-stable/manuals/v.db.select.html))
 * viewshed computation ([r.viewshed](https://grass.osgeo.org/grass-stable/manuals/r.viewshed.html))
 * simple parallelization ([multiprocessing.Pool](https://docs.python.org/3/library/multiprocessing.html))
 * region handling ([grass.script.region_env](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.region_env))
 * raster algebra ([r.mapcalc](https://grass.osgeo.org/grass-stable/manuals/r.mapcalc.html))
 * temporal data handling
 * reprojecting ([r.proj](https://grass.osgeo.org/grass-stable/manuals/r.proj.html))
 * raster mask ([r.mask](https://grass.osgeo.org/grass-stable/manuals/r.mask.html))
 * raster as numpy array ([grass.script.array](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#module-script.array))

In the previous notebook (Part 1) we created new location (project) *dix_park*. This automatically created new mapset (subproject) _PERMANENT_ where we then imported our base data. Now it's time to create a new mapset for our viewshed analysis, we will name it _viewshed_:

In [ ]:
%%bash
grass -c -e ~/grassdata/dix_park/viewshed

In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys
from tqdm import tqdm

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/grassdata", "dix_park", "viewshed")

## Data preparation
We will first derive viewpoints along the road *Umstead Drive* (vector `umstead_drive_segments`) that we extracted in the first part of the workshop.

1. Because the road consists of several segments, we will merge them into one.
2. Create new vector of points along the line with distance 50 m.

In [ ]:
gs.run_command("v.build.polylines", input="umstead_drive_segments", output="umstead_drive", cats="first")
gs.run_command("v.to.points", input="umstead_drive", type="line", output="viewpoints", dmax=50)

Visualize the points with InteractiveMap with OSM tiles (see [other tile options](https://grass.osgeo.org/grass-stable/manuals/libpython/grass.jupyter.html#module-grass.jupyter.interactivemap)):

In [ ]:
road_map = gj.InteractiveMap(tiles="OpenStreetMap")
road_map.add_vector("umstead_drive")
road_map.add_vector("viewpoints")
road_map.show()

Next part of analysis is raster-based, so we need to make sure we set computational region as we need. Specifically, we set it to match the DSM:

In [ ]:
gs.run_command("g.region", raster="dsm")

Now we want to compute the visibility using DSM, however some points may fall on top of a tree, so we need to filter those out.

First compute height above ground (DSM - DTM):

In [ ]:
gs.mapcalc("diff = dsm - ground")
gs.run_command("r.colors", map="diff", color="differences")

diff_map = gj.Map()
diff_map.d_rast(map="diff")
diff_map.d_vect(map="umstead_drive")
diff_map.d_legend(raster="diff")
diff_map.show()

Extract height above ground for the viewpoint locations to identify points that fall on top of a tree growing next to the road:

In [ ]:
gs.run_command("v.what.rast", map="viewpoints", layer=2, raster="diff", column="height")

See the newly computed attribute data. This example shows how the attribute data can be loaded into pandas:

In [ ]:
import json
import pandas as pd
pd.DataFrame(json.loads(gs.read_command("v.db.select", map="viewpoints", columns="cat,height", layer=2, format="json"))["records"])

Visualize the viewpoints with the height-above-ground raster. You can filter the points based on the height above ground, we won't display points with height > 2.
Additionally, we will render the result larger (`width=1000`) and we will render the map zoomed in to the area with the points
by saving a region and using it in Map (`saved_region="umstead_drive_region"`).

In [ ]:
gs.run_command("g.region", vector="umstead_drive", align="dsm", save="umstead_drive_region")

img = gj.Map(width=1000, saved_region="umstead_drive_region")
img.d_rast(map="diff")
img.d_vect(map="umstead_drive")
img.d_vect(map="viewpoints", layer=2, where="height >= 2", size=15, icon="basic/pin", fill_color="red")
img.d_vect(map="viewpoints", layer=2, where="height < 2", size=15, icon="basic/pin")
img.d_legend(raster="diff")
img.show()

## Viewshed computation
To get the cumulative viewshed, we will compute viewsheds from all the viewpoints we generated earlier.
First, we get the list coordinates of the viewpoints that are likely lying on the ground:

In [ ]:
viewpoints = gs.read_command('v.out.ascii', input='viewpoints',
                             separator='comma', layer=2, where="height < 2").strip().splitlines()
viewpoints = [p.split(",") for p in viewpoints]
viewpoints

We will now compute the viewshed from each viewpoint in a loop. We set max distance of 300 m. Each viewshed will be named `viewshed_cat`.

In [ ]:
%%time
maps = []
for x, y, cat in tqdm(viewpoints):
    name = f"viewshed_{cat}"
    gs.run_command("r.viewshed", input="dsm", output=name,
                   coordinates=(x, y), max_distance=300, flags="b")
    maps.append(name)

Since these are independent runs, we can easily parallelize the r.viewshed calls using Python multiprocessing.
We define a function that computes the viewshed and returns the name of the output or None in case of error:

In [ ]:
%%time
from grass.exceptions import CalledModuleError
from multiprocessing import Pool, cpu_count


def viewshed(point):
    x, y, cat = point
    x, y = float(x), float(y)
    name = f"viewshed_{cat}"
    try:
        gs.run_command("r.viewshed", input="dsm", output=name,
                       coordinates=(x, y), max_distance=300, flags="b")
        return f"viewshed_{cat}"
    except CalledModuleError:
        return None

# run with the number of CPUs available
# proc = cpu_count()
proc = 1
with Pool(processes=proc) as pool:
    maps = pool.map_async(viewshed, viewpoints).get()
print(maps)

One trick to speedup viewshed computation is to limit the computation only
to the actual area given by the maxdistance option. To do that we will locally modify the computational region
and pass the environment to the module directly. The current computational region won't be affected.

In [ ]:
%%time
import os
from grass.exceptions import CalledModuleError
from multiprocessing import Pool, cpu_count


def viewshed(point):
    x, y, cat = point
    x, y = float(x), float(y)
    max_distance = 300
    # copy current environment
    env = os.environ.copy()
    # set GRASS_REGION variable using region_env function
    env["GRASS_REGION"] = gs.region_env(align="dsm",
                                        e=x + max_distance,
                                        w=x - max_distance,
                                        n=y + max_distance,
                                        s=y - max_distance)
    name = f"viewshed_{cat}"
    try:
        gs.run_command("r.viewshed", input="dsm", output=name, flags="b",
                      coordinates=(x, y), max_distance=max_distance, env=env)
        return f"viewshed_{cat}"
    except CalledModuleError:
        return None

# run with the number of CPUs available
# proc = cpu_count()
proc = 1
with Pool(processes=proc) as pool:
    maps = pool.map_async(viewshed, viewpoints).get()
print(maps)
print(f"Viewshed num cells: {gs.raster_info(maps[0])['cells']}")
print(f"DSM num cells: {gs.raster_info('dsm')['cells']}")


## Temporal dataset of viewsheds

In this part we will create, analyze and visualize a temporal dataset of viewsheds using [temporal tools](https://grass.osgeo.org/grass-stable/manuals/temporal.html). 

First, let's check we have the viewshed rasters ready:

In [ ]:
gs.list_strings(type="raster", pattern="viewshed_*")

We will create an empty space-time raster dataset called _viewsheds_ with relative temporal type:

In [ ]:
gs.run_command("t.create", output="viewsheds", type="strds", temporaltype="relative",
              title="Viewshed series", description="Series of viewsheds along a road")

Now we register the viewshed rasters with start time 1 and 1-minute increment to simulate a change of view of a car driving slowly along the road:

In [ ]:
gs.run_command("t.register", input="viewsheds", maps=",".join(maps), start=1, unit="minutes", increment=1)

Let's print basic dataset info. We will use this info later on to set computational region covering the entire dataset.

In [ ]:
from pprint import pprint
info = gs.parse_command("t.info", input="viewsheds", flags="g")
pprint(info)

To list the individual rasters, we will use t.rast.list. Notice there is no end time, because we don't use interval data.

In [ ]:
from io import StringIO

pd.read_csv(StringIO(gs.read_command("t.rast.list", input="viewsheds", separator="comma", columns="name,start_time,end_time")))

We can quickly get basic statistics such as the size of the viewsheds (see _sum_ column for the number of visible cells):

In [ ]:
df = pd.read_csv(StringIO(gs.read_command("t.rast.univar", input="viewsheds", separator="comma")))
df

Let's find and visualize largest and smallest viewshed:

In [ ]:
largest = df.iloc[df[['sum']].idxmax()["sum"]].id
smallest = df.iloc[df[['sum']].idxmin()["sum"]].id

gs.run_command("g.region", raster=[largest, smallest], save="zoom_region")
viewshed_map = gj.Map(saved_region="zoom_region")
viewshed_map.d_rast(map="ortho")
viewshed_map.d_rast(map=largest, values=1)
viewshed_map.d_rast(map=smallest, values=1)
viewshed_map.d_vect(map="umstead_drive", color="white")
viewshed_map.show()

Let's compute a temporal dataset where values of each viewshed will represent the registered start time.

Before we do that, let's set computational region to match the bounding box of the entire dataset, here we use previous t.info output:

In [ ]:
gs.run_command("g.region", n=info["north"], s=info["south"], e=info["east"], w=info["west"])

We use temporal raster algebra. Here we compute a new temporal dataset _viewsheds_start_ so that for example viewshed with start time 5 has value 5 for visible area and no data for invisible area.

In [ ]:
gs.run_command("t.rast.mapcalc", inputs="viewsheds", output="viewsheds_start", basename="viewshed_start",
               expression="if (viewsheds == 0, null(), start_time())")

Set color of the newly computed time series:

In [ ]:
gs.run_command("t.rast.colors", input="viewsheds_start", color="plasma")

With TimeSeriesMap, we can interactively visualize the time series:

In [ ]:
timemap = gj.TimeSeriesMap(width=800)
timemap.d_rast(map="ortho")
timemap.d_vect(map="umstead_drive", )
timemap.add_raster_series("viewsheds_start")
timemap.show()

We can export an animated GIF:

In [ ]:
from IPython.display import Image

Image(timemap.save("animation.gif", duration=300))

## Cumulative viewshed
We can compute the cumulative viewshed, which aggregates viewsheds from multiple viewpoints. In this way you can e.g., identify the most frequently visible areas from the road.


Since our viewshed rasters are binary (0 invisible, 1 visible), we will use r.series method *sum*. Then we replace zeros with no data using r.null and set a new color ramp:

In [ ]:
# cumulative viewshed
gs.run_command("r.series", input=maps, output="cumulative_viewshed", method="sum")
gs.run_command("r.null", map="cumulative_viewshed", setnull=0)
gs.run_command("r.colors", map="cumulative_viewshed", color="plasma")

Let's visualize the results:

In [ ]:
cumulative_map = gj.InteractiveMap()
cumulative_map.add_raster("cumulative_viewshed", opacity=0.8)
cumulative_map.add_vector("umstead_drive")
cumulative_map.add_layer_control(position="bottomright")
cumulative_map.show()

And create a 3D rendering with draped cumulative viewshed over the DSM:

In [ ]:
map3d = gj.Map3D()
map3d.render(elevation_map="dsm", resolution_fine=1, color_map="cumulative_viewshed",
           vline="umstead_drive", vline_width=3, vline_color="white", light_brightness=50,
           position=[0.4, 0.8], height=3000, perspective=10)
map3d.overlay.d_legend(raster="cumulative_viewshed", at=(0, 30, 1, 7), use=[1, 2, 3, 4, 5, 6], flags="fb")
map3d.show()

## Data reprojection
Next, we will analyze the cumulative viewshed to see how much greenery a driver would see on the way. To do that we compute NDVI:

We reproject R and NIR Landsat bands from NCSPM sample dataset we already have available. Tool r.proj respects the current region (extent and resolution), but you can set resolution to certain value, we use 28.5 m which is the original resolution.


In [ ]:
gs.run_command("g.region", raster="dsm")
for band in [30, 40]:
    gs.run_command("r.proj", location="nc_spm_08_grass7", mapset="PERMANENT", input=f"lsat7_2002_{band}", method="nearest", resolution=28.5)

Compute NDVI:

In [ ]:
gs.run_command("i.vi", viname="ndvi", red="lsat7_2002_30", nir="lsat7_2002_40", output="ndvi")

In [ ]:
ndvi_map = gj.Map()
ndvi_map.d_rast(map="ndvi")
ndvi_map.d_legend(raster="ndvi")
ndvi_map.show()

## Mask
Now let's analyze what is the distribution of NDVI within the visible area. We will mask the data by the visible area:


In [ ]:
gs.run_command("r.mask", raster="cumulative_viewshed")
data = gs.parse_command("r.univar", map="ndvi", flags="g")
print(f"Average NDVI of visible cells: {float(data['mean']):.2f} ± {float(data['stddev']):.2f}")

In [ ]:
ndvi_map = gj.Map()
ndvi_map.d_rast(map="ndvi")
ndvi_map.d_legend(raster="ndvi", flags="d")
ndvi_map.show()

Let's see the histogram of visible NDVI using d.histogram:

In [ ]:
histo = gj.Map(width=800, height=400)
histo.d_histogram(map="ndvi", bgcolor="grey")
histo.show()

## Read as numpy array
It is also easy to use the results as a numpy array and then use other Python libraries to analyze the data:

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from grass.script import array as garray

In [ ]:
ndvi = garray.array(mapname="ndvi", null='nan')
ndvi

In [ ]:
sns.set_style('darkgrid')
sns.histplot(ndvi.ravel(), kde=True)

Finally, remove the mask:

In [ ]:
gs.run_command("r.mask", flags="r")

# Part 3: Estimating inundation extent using HAND methodology


In this example we will use some of GRASS GIS hydrology tools, namely:

* [r.watershed](https://grass.osgeo.org/grass-stable/manuals/r.watershed.html): for computing flow accumulation, drainage direction, the location of streams and watershed basins; it does not need sink filling because of using the least-cost-path to route flow out of sinks
* [r.lake](https://grass.osgeo.org/grass-stable/manuals/r.lake.html): to fill a lake to a target water level from a given start point or seed raster
* [r.lake.series](https://grass.osgeo.org/grass-stable/manuals/r.lake.series.html): addon which runs r.lake for different water levels
* [r.stream.distance](https://grass.osgeo.org/grass-stable/manuals/r.stream.distance.html): for computing the distance to streams or outlet, the relative elevation above streams; the distance and the elevation are calculated along watercourses

First, let's create a new mapset *flooding* in nc_spm_08_grass7 sample dataset:

In [ ]:
%%bash
grass -c -e ~/grassdata/nc_spm_08_grass7/flooding

Initialize GRASS session:

In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/grassdata", "nc_spm_08_grass7", "flooding")

Modules r.stream.distance and r.lake.series are addons and we need to install them first:

In [ ]:
gs.run_command("g.extension", extension="r.stream.distance")
gs.run_command("g.extension", extension="r.lake.series")

## Compute HAND raster

We will estimate inundation extent using the Height Above Nearest Drainage methodology ([A.D. Nobre, 2011](https://doi.org/10.1016/j.jhydrol.2011.03.051)). We will compute the HAND terrain model representing the differences in elevation between each grid cell and the elevations of the flowpath-connected downslope grid cells where the flow enters the channel.

First we compute the flow accumulation, drainage and streams (with a threshold value of 100000). We convert the streams to vector for better visualization.

In [ ]:
gs.run_command("g.region", raster="elevation")
gs.run_command("r.watershed", elevation="elevation", accumulation="flowacc",
               drainage="drainage", stream="rwatershed_streams", threshold=100000)
gs.run_command("r.to.vect", input="rwatershed_streams", output="rwatershed_streams", type="line")

fllowacc_map = gj.Map()
fllowacc_map.d_rast(map="flowacc")
fllowacc_map.d_vect(map="rwatershed_streams", width=2, color="blue")
fllowacc_map.d_legend(raster="flowacc", range="0,1000")
fllowacc_map.show()

Let's zoom in to see the flow accumulation raster better:

In [ ]:
flowacc_map = gj.InteractiveMap()
flowacc_map.add_raster("flowacc")
flowacc_map.add_vector("rwatershed_streams")
flowacc_map.show()

Now we use r.stream.distance with output parameter difference to compute a new raster map where each cell is the elevation difference between the cell and the the cell on the stream where the cell drains.

In [ ]:
gs.run_command("r.stream.distance", stream_rast="rwatershed_streams", direction="drainage",
               elevation="elevation", method="downstream", difference="above_stream")
gs.run_command("r.colors", map="above_stream", color="elevation")

Compare original elevation and HAND raster:

In [ ]:
map3d = gj.Map3D()
map3d.render(elevation_map="elevation", color_map="elevation",
             position=(0.5, 1), height=3000, perspective=12, zexag=5)
map3d.overlay.d_legend(raster="elevation", at=(5, 80, 87, 92), flags="b", border_color="none")
map3d.show()

In [ ]:
map3d = gj.Map3D()
map3d.render(elevation_map="above_stream", color_map="above_stream",
             position=(0.5, 1), height=3000, perspective=12, zexag=5)
map3d.overlay.d_legend(raster="above_stream", at=(5, 80, 90, 95), flags="b", border_color="none")
map3d.show()

## Inundation
Before we compute the inundation, we will look at how r.lake works. We compute a lake from a specified coordinate pair and water level:

In [ ]:
gs.run_command("r.lake", elevation="elevation", water_level=90, lake="lake", coordinates=[637877, 218475])

lake_map = gj.Map()
lake_map.d_rast(map="elevation")
lake_map.d_rast(map="lake")
lake_map.d_legend(raster="lake", label_values="0.1,5,10,15", digits=0, at=(1, 40, 90, 95))
lake_map.show()

Now instead of the elevation raster we use the HAND raster to simulate 5-meter inundation and, as the seed we specify the entire stream.

In [ ]:
gs.run_command("r.lake", elevation="above_stream", water_level=5, lake="flood", seed="rwatershed_streams")

hand_map = gj.Map()
hand_map.d_rast(map="above_stream")
hand_map.d_rast(map="flood")
hand_map.d_legend(raster="flood", label_values="0.1,4,8", digits=0, at=(1, 40, 90, 95))
hand_map.show()

With the r.lake.series addon we can create a series of inundation maps with rising water levels:

In [ ]:
from io import StringIO
import pandas as pd

gs.run_command("r.lake.series", elevation="above_stream", start_water_level=0, end_water_level=5, 
               water_level_step=0.5, output="inundation", seed_raster="rwatershed_streams", quiet=True)
gs.run_command("t.rast.colors", input="inundation", color="water")
pd.read_csv(StringIO(gs.read_command("t.rast.list", input="inundation", separator="comma")))

r.lake.series creates a space-time dataset. We can use the [temporal modules](https://grass.osgeo.org/grass-stable/manuals/temporal.html) to further work with the data. For example, we could further compute the volume and extent of flood water using t.rast.univar:

In [ ]:
pd.read_csv(StringIO(gs.read_command("t.rast.univar", input="inundation", separator="comma")))

Let's visualize the results:

In [ ]:
timemap = gj.TimeSeriesMap()
timemap.d_rast(map="elevation_shade")
timemap.d_vect(map="streets_wake", color="#4D4D4D")
timemap.add_raster_series("inundation")
timemap.d_legend(at=(1, 40, 90, 95))
timemap.show()

# Part 4: Image segmentation


In this example we will show segmentation of a Landsat scene.
We show two segmentation modules: [i.segment](https://grass.osgeo.org/grass-stable/manuals/i.segment.html) and the addon [i.superpixels.slic](https://grass.osgeo.org/grass-stable/manuals/addons/i.superpixels.slic.html).
Note that each segmentation algorithm is designed for different purpose, so we can't directly compare them.

First, let's create a new mapset *segmentation* in nc_spm_08_grass7 location:

In [ ]:
%%bash
grass -c -e ~/grassdata/nc_spm_08_grass7/segmentation

Initialize GRASS session:

In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/grassdata", "nc_spm_08_grass7", "segmentation")

Install the addon:

In [ ]:
gs.run_command("g.extension", extension="i.superpixels.slic")

## Data preparation
Imagery modules typically work with *imagery groups*. We first list the landsat raster data and then create an imagery group:



In [ ]:
maps = gs.list_grouped(type="raster", pattern="lsat*")["PERMANENT"]
print(maps)
gs.run_command("i.group", group="landsat", subgroup="landsat", input=maps)

Next, we derive NDVI to see some of the effects of segmentation:

In [ ]:
gs.run_command("g.region", raster="lsat7_2002_30")
gs.run_command("i.vi", red="lsat7_2002_30",  nir="lsat7_2002_40", output="ndvi", viname="ndvi")

ndvi_map = gj.Map()
ndvi_map.d_rast(map="ndvi")
ndvi_map.d_legend(raster="ndvi", at=(2, 50, 1, 5))
ndvi_map.show()

### Segmentation with i.superpixels.slic
Superpixels can be defined as a group of pixels that share common characteristics and are useful in computer vision and image processing.
Here we run [i.superpixels.slic](https://grass.osgeo.org/grass-stable/manuals/addons/i.superpixels.slic.html) and convert the resulting raster to vector for better visualization.

In [ ]:
gs.run_command("i.superpixels.slic", input="landsat", output="superpixels", num_pixels=1000, compactness=0.5)
gs.run_command("r.to.vect", input="superpixels", output="superpixels", type="area")

You can play with *compactness* and *num_pixels* parameters and see how the resulting segmentation changes:

In [ ]:
superpixels_map = gj.Map()
superpixels_map.d_rast(map="ndvi")
superpixels_map.d_vect(map="superpixels", width=1, color="black", fill_color="none")
superpixels_map.d_legend(raster="ndvi", at=(2, 50, 1, 5))
superpixels_map.show()

For fun, let's do zonal statistics on the results. We compute the median NDVI value within each segment:

In [ ]:
gs.run_command("r.stats.quantile", base="superpixels", cover="ndvi", output="superpixels_ndvi")

superpixels_map = gj.Map()
superpixels_map.d_rast(map="superpixels_ndvi")
superpixels_map.d_legend(raster="superpixels_ndvi", at=(2, 50, 1, 5))
superpixels_map.show()

### Segmentation with i.segment

Next, we do the same, but with i.segment to see the different behavior. Note that i.segment uses *region growing* algorithm by default, but *mean shift* is also available:

In [ ]:
gs.run_command("i.segment", group="landsat", output="segments", threshold=0.5, minsize=50)
gs.run_command("r.to.vect", input="segments", output="segments", type="area")

In [ ]:
segments_map = gj.Map()
segments_map.d_rast(map="ndvi")
segments_map.d_vect(map="segments", width=1, color="black", fill_color="none")
segments_map.d_legend(raster="ndvi", at=(2, 50, 1, 5))
segments_map.show()

In [ ]:
gs.run_command("r.stats.quantile", base="segments", cover="ndvi", output="segments_ndvi")

segments_map = gj.Map()
segments_map.d_rast(map="segments_ndvi")
segments_map.d_legend(raster="superpixels_ndvi", at=(2, 50, 1, 5))
segments_map.show()

# Part 5: Parallelization examples

This part will briefly cover how to run GRASS computations in parallel. First, create a new mapset:

In [ ]:
%%bash
grass -c -e ~/grassdata/nc_spm_08_grass7/parallelization

In [ ]:
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/grassdata", "nc_spm_08_grass7", "parallelization")

## Tool-level parallelization
There are several [internally parallelized tools](https://grass.osgeo.org/grass-stable/manuals/keywords.html#parallel), either using OpenMP or Python multiprocessing library. We can use `nprocs` option to set the number of cores to be used for processing.


Set computational region to match _elevation_ raster.

In [ ]:
gs.run_command("g.region", raster="elevation")

Compute moving window analysis and measure time first with one core, than with 2:

In [ ]:
%%timeit
gs.run_command("r.neighbors", input="elevation", output="elevation_smoothed", method="average", size=25, nprocs=1)

In [ ]:
%%timeit
gs.run_command("r.neighbors", input="elevation", output="elevation_smoothed", method="average", size=25, nprocs=2)

Visualize original and smoothed raster (turn layers on and off):

In [ ]:
neighbors_map = gj.InteractiveMap()
neighbors_map.add_raster("elevation")
neighbors_map.add_raster("elevation_smoothed")
neighbors_map.add_layer_control(position="bottomright")
neighbors_map.show()

## GridModule for tiling
Some compute-intensive tasks can benefit from spatially splitting the task into tiles, and then running the task in parallel. [GridModule](https://grass.osgeo.org/grass-stable/manuals/libpython/pygrass.modules.grid.html) can automate this splitting-computing-merging procedure and execute the computation in parallel.

In this example, we will interpolate an elevation surface from vector points using IDW interpolation. First, set computational region to match the extent of vector points and set the resolution to 1 meter:

In [ ]:
gs.run_command("g.region", vector="elev_lid792_bepts", res=1, flags="a")

Measure the time without using GridModule:

In [ ]:
%%timeit
gs.run_command("v.surf.idw", input="elev_lid792_bepts", output="elev_lid792_interp")

And now with GridModule:

In [ ]:
%%writefile interpolation.py
from grass.pygrass.modules.grid import GridModule


grid = GridModule(
    "v.surf.idw",
    input="elev_lid792_bepts",
    output="elev_lid792_interp",
    processes=3,
    overlap=12,
    quiet=True,
)
grid.run()

In [ ]:
%%timeit
!python interpolation.py

In [ ]:
gs.run_command("r.colors", map="elev_lid792_interp", color="elevation")
neighbors_map = gj.Map()
neighbors_map.d_rast(map="elev_lid792_interp")
neighbors_map.d_vect(map="elev_lid792_bepts", size=1, color="black")
neighbors_map.show()

## Running multiple independent computations
In this example, our goal is to compute multiple viewsheds and export them as picture. Since these are independent computations, we can run them in parallel.
The first part implements this task in Python using _multiprocessing_ library (also explained in Part 2)
and the second part will run each computation using `grass --exec` interface in separate mapsets that allows us to potentially distribute the computation across multiple nodes on an HPC.

First compute a shaded relief raster for visualization:

In [ ]:
gs.run_command("g.region", raster="elevation")
gs.run_command("r.relief", input="elevation", output="relief")

We will compute viewsheds from vector points _firestations_:

In [ ]:
viewpoints = gs.read_command('v.out.ascii', input='firestations', separator='comma', flags="r").strip().splitlines()
viewpoints = [p.split(",") for p in viewpoints]
viewpoints

We will extend script from Part 2 of this workshop to include rendering to file. Set `nprocs` to more than 1 when possible.

In [ ]:
import os
from grass.exceptions import CalledModuleError
from multiprocessing import Pool, cpu_count


def viewshed(point):
    x, y, cat = point
    x, y = float(x), float(y)
    max_distance = 2000
    # copy current environment
    env = os.environ.copy()
    # set GRASS_REGION variable using region_env function
    env["GRASS_REGION"] = gs.region_env(align="elevation",
                                        e=x + max_distance,
                                        w=x - max_distance,
                                        n=y + max_distance,
                                        s=y - max_distance)
    name = f"viewshed_{cat}"
    try:
        gs.run_command("r.viewshed", input="elevation", output=name, flags="b",
                      coordinates=(x, y), max_distance=max_distance, env=env)
        # create visualization
        viewshed_map = gj.Map(use_region=True, env=env)
        viewshed_map.d_rast(map="relief")
        viewshed_map.d_rast(map=f"viewshed_{cat}", values=1)
        viewshed_map.d_vect(map="firestations", cat=cat, size=15, icon="basic/pin")
        viewshed_map.save(f"viewshed_{cat}.png")
        return f"viewshed_{cat}"
    except CalledModuleError:
        return None

# run with the number of CPUs available
# proc = cpu_count()
nprocs = 1
with Pool(processes=nprocs) as pool:
    maps = pool.map_async(viewshed, viewpoints).get()

Let's look at one of the computed and rendered viewsheds:

In [ ]:
from IPython.display import Image

Image("viewshed_22.png")

Note that this way, we can't distribute the computation across multiple nodes (hundreds of cores).
We will do the same thing differently, using `grass --exec` [interface](https://grass.osgeo.org/grass-stable/manuals/grass.html), running each task in a separate mapset. This way, the tasks could be distributed across multiple nodes.

`--exec` interface allows GRASS tools and user scripts to be executed in a GRASS GIS non-interactive session. For example, here is a simple call to list all available vectors in PERMANENT mapset:

In [ ]:
%%bash
grass ~/grassdata/nc_spm_08_grass7/PERMANENT --exec g.list type=vector mapset=PERMANENT -t

Now we will create a Python script `myscript.py` computing and rendering viewsheds similarly as in the previous example. The script requires 3 parameters (x and y coordinate, and category). Note that we can set computational region in a standard way, because each script will run in separate mapset, so the different regions won't interfere with each other.

In [ ]:
%%writefile myscript.py
import sys
import grass.script as gs
import grass.jupyter as gj


def main(x, y, cat):
    max_distance = 2000
    x, y = float(x), float(y)
    name = f"viewshed_{cat}"
    gs.run_command("g.region", align="elevation", e=x + max_distance,
                   w=x - max_distance, n=y + max_distance, s=y - max_distance)
    gs.run_command("r.viewshed", input="elevation", output=name, coordinates=(x, y),
                   observer_elevation=3, max_distance=max_distance, flags="b")
    # create visualization
    viewshed_map = gj.Map(use_region=True)
    viewshed_map.d_rast(map="relief@parallelization")
    viewshed_map.d_rast(map=f"viewshed_{cat}", values=1)
    viewshed_map.d_vect(map="firestations", cat=cat, size=15, icon="basic/pin")
    viewshed_map.save(f"viewshed_{cat}.png")

if __name__ == "__main__":
    args = sys.argv[1:]
    main(*args)

We will generate a file `jobs.sh` with one command per line. We will run each task in a temporary mapset so all computed data will be deleted afterwards. That is fine for our example where we need only the final PNG files.

In [ ]:
with open("jobs.sh", "w") as f:
    for viewpoint in viewpoints:
        f.write(f"grass --tmp-mapset ~/grassdata/nc_spm_08_grass7 --exec python myscript.py {viewpoint[0]} {viewpoint[1]} {viewpoint[2]}\n")

This is the content of the file:

In [ ]:
!cat jobs.sh

To execute these commands in parallel, we can use e.g. [GNU Parallel](https://www.gnu.org/software/parallel/):

In [ ]:
%%bash

parallel -j 2 < jobs.sh

Check one of the resulting PNG files:

In [ ]:
Image("viewshed_22.png")